Annie Taylor
5.31.22

### Combine MESS Maps
SAHM runs a Multivariate Environmental Similarity Surfaces (MESS) analysis on each model, which reveal parts of the study area where that model may be extrapolating beyond the range of values it was trained on. This script combines the MESS maps across each model for each species. 


#### Import packages

In [79]:
import arcpy
import os
import pandas as pd
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

#### Set Parameters

In [80]:
# The species to investigate
# codes = ['CHPO','QUKE','RUUR']
codes = ['QUKE']

# folders to iterate over/into, automatically created by SAHM based on models that you choose
folders = ['brt_1', 'glm_1', 'mars_1', 'rf_1', 'Maxent_1']
models = ['brt', 'glm', 'mars', 'rf', 'Maxent']

# get two main projections and have study area outlines in each of those projections
studyarea_wgs = r"D:/1_AMLT/1_SDM/SDM_Pro/SDM.gdb/AMLT_StewardshipArea_UTM_10kmWGS84"
wgssr = arcpy.Describe(studyarea_wgs).spatialReference

studyarea_utm = r"D:/1_AMLT/1_SDM/SDM_Pro/SDM.gdb/AMLT_StewardshipArea_UTM_10km"    
utmsr = arcpy.Describe(studyarea_utm).spatialReference

#### Get MESS Maps for each model run, reclassify to negative (extrapolation) and positive (fits training data) values

In [82]:
for code in codes:
    results = r"D:/1_AMLT/1_SDM/SAHM/" + code + "/"
    # create a list of all of the MESS rasters
    rasters = []

    # save binary reclassifications of mess maps so that 1 = extrapolation, 0 = no problem
    for model in models:
        mess = results + model + '_1/' + model + '_mess_map.tif'
        binmess = results + model + '_1/' + model + '_bin_mess_map.tif'
       
        if os.path.exists(binmess):
            print('File ' + binmess + ' already exists')
        else:
            print('Reclassifying to ' + binmess)
            outras = arcpy.sa.Reclassify(mess, "VALUE", "-1000 -0.000010 1;-0.000010 1000 0", "DATA")
            outras.save(binmess)
        rasters.append(binmess)
    
    # now add them together! 
    # initialize outras as the first raster in the list so that this resets each time I run the cell
    outras = arcpy.Raster(rasters[0])
    # iterate over the other rasters
    for raster in rasters[1:]:
        # Cast file name as raster to enable spatial analyst addition
        listras = arcpy.Raster(raster)
        # Add each raster to an output raster
        outras += listras
    # Save combined raster to file
    outmess = results + code + '_All_MESS.tif'
    outmessclip = results + code + '_All_MESS_Clip.tif'

    # if you don't want to overwrite files use this
    if os.path.exists(outmess):
        print('File ' + outmess + ' already exists')
    else:
        print('Saving to ' + outmess)
        outras.save(outmess)
    # if you want to overwrite the file use this
    # outras.save(outwgs)

    # Save a clipped version of it
    if os.path.exists(outmessclip):
        print('File ' + outmessclip + ' already exists')
    else:
        print('Saving to ' + outmessclip)
        arcpy.management.Clip(outmess, "-122.476320553 36.138630725 -120.367393001 37.328780964", outmessclip, studyarea_wgs, clipping_geometry="ClippingGeometry", maintain_clipping_extent="MAINTAIN_EXTENT")


File D:/1_AMLT/1_SDM/SAHM/QUKE/brt_1/brt_bin_mess_map.tif already exists
File D:/1_AMLT/1_SDM/SAHM/QUKE/glm_1/glm_bin_mess_map.tif already exists
File D:/1_AMLT/1_SDM/SAHM/QUKE/mars_1/mars_bin_mess_map.tif already exists
File D:/1_AMLT/1_SDM/SAHM/QUKE/rf_1/rf_bin_mess_map.tif already exists
File D:/1_AMLT/1_SDM/SAHM/QUKE/Maxent_1/Maxent_bin_mess_map.tif already exists
File D:/1_AMLT/1_SDM/SAHM/QUKE/QUKE_All_MESS.tif already exists
File D:/1_AMLT/1_SDM/SAHM/QUKE/QUKE_All_MESS_Clip.tif already exists
